In [1]:
import numpy as np
import gurobipy as gp
import pandas as pd

In [19]:
ind_data = pd.read_csv('index_data.csv')
roi_data = pd.read_csv('ROI_data.csv')
roi_mat = pd.read_csv('roi_mat.csv')

In [42]:
first_roi = roi_data.iloc[0,1:]

In [43]:
MarkMod=gp.Model()
MarkModX = MarkMod.addMVar(10)

In [44]:
MarkMod.setObjective(gp.quicksum(first_roi[i]*MarkModX[i] for i in range(10)),sense=gp.GRB.MAXIMIZE)

In [45]:
conlist=[0]*4
conlist[0] = MarkMod.addConstr(MarkModX[0] + MarkModX[1] <= MarkModX[4] + MarkModX[9])
conlist[1] = MarkMod.addConstr(MarkModX[4] + MarkModX[5] + MarkModX[6] + MarkModX[7] + MarkModX[8] >= 2*(MarkModX[2] + MarkModX[3]))
vari = MarkMod.addConstrs((MarkModX[i] <=3) for i in range(10))
conlist[3] = MarkMod.addConstr(gp.quicksum(MarkModX[i]for i in range(10)) <= 10)

In [46]:
MarkMod.Params.OutputFlag = 0
MarkMod.optimize() # solve the model
MarkMod.objVal # print the optimal objective value

0.45600000000000007

In [72]:
MarkModX.x # print the optimal decision variables

array([0., 3., 0., 1., 0., 0., 3., 0., 0., 3.])

#### Q2

In [54]:
sec_roi = roi_data.iloc[1,1:]
sec_roi

Print        0.049
TV           0.023
SEO          0.024
AdWords      0.039
Facebook     0.044
LinkedIn     0.046
Instagram    0.026
Snapchat     0.019
Twitter      0.037
Email        0.026
Name: 1, dtype: object

In [55]:
MarkMod2=gp.Model()
MarkMod2X = MarkMod2.addMVar(10)

In [56]:
MarkMod2.setObjective(gp.quicksum(sec_roi[i]*MarkMod2X[i] for i in range(10)),sense=gp.GRB.MAXIMIZE)

In [57]:
conlist2=[0]*4
conlist2[0] = MarkMod2.addConstr(MarkMod2X[0] + MarkMod2X[1] <= MarkMod2X[4] + MarkMod2X[9])
conlist2[1] = MarkMod2.addConstr(MarkMod2X[4] + MarkMod2X[5] + MarkMod2X[6] + MarkMod2X[7] + MarkMod2X[8] >= 2*(MarkMod2X[2] + MarkMod2X[3]))
vari2 = MarkMod2.addConstrs((MarkMod2X[i] <=3) for i in range(10))
conlist2[3] = MarkMod2.addConstr(gp.quicksum(MarkMod2X[i]for i in range(10)) <= 10)

In [58]:
MarkMod2.Params.OutputFlag = 0
MarkMod2.optimize() # solve the model
MarkMod2.objVal # print the optimal objective value

0.45600000000000007

In [59]:
MarkMod2X.x # print the optimal decision variables

array([3., 0., 0., 1., 3., 3., 0., 0., 0., 0.])

### Q5 - Do you think the third constraint above, based on your boss’ experience, is useful? - NEED TO WRITE INSIGHTS HERE

In [53]:
[vari[i].Pi for i in range(10)]

[array(0.),
 array(0.),
 array(0.),
 array(0.),
 array(0.),
 array(0.),
 array(0.007),
 array(0.),
 array(0.),
 array(0.015)]

Make sense to add because 2 constraints are binding

### Q6 - NEED TO WRITE INSIGHTS HERE

In [89]:
first_roi

Print        0.031
TV           0.049
SEO          0.024
AdWords      0.039
Facebook     0.016
LinkedIn     0.024
Instagram    0.046
Snapchat     0.026
Twitter      0.033
Email        0.044
Name: 0, dtype: object

In [60]:
MarkModX.SAObjLow

array([ -inf, 0.039,  -inf, 0.033,  -inf,  -inf, 0.039,  -inf,  -inf,
       0.029])

In [61]:
MarkModX.SAObjUp

array([0.049, 0.062, 0.039, 0.046, 0.029, 0.039,   inf, 0.039, 0.039,
         inf])

In [63]:
sec_roi

Print        0.049
TV           0.023
SEO          0.024
AdWords      0.039
Facebook     0.044
LinkedIn     0.046
Instagram    0.026
Snapchat     0.019
Twitter      0.037
Email        0.026
Name: 1, dtype: object

In [64]:
MarkMod2X.SAObjLow

array([0.039 ,   -inf,   -inf, 0.0375, 0.029 , 0.039 ,   -inf,   -inf,
         -inf,   -inf])

In [65]:
MarkMod2X.SAObjUp

array([0.052, 0.049, 0.039, 0.046,   inf,   inf, 0.039, 0.039, 0.039,
       0.029])

### Q7

In [66]:
roi_mat.head()

,Unnamed: 0,Print,TV,SEO,AdWords,Facebook,LinkedIn,Instagram,Snapchat,Twitter,Email
0,January,4.0,3.6,2.4,3.9,3.0,3.5,3.6,2.25,3.5,3.5
1,February,4.0,3.9,2.7,3.8,4.3,3.2,2.7,1.80,3.7,3.5
2,March,3.5,2.9,3.1,3.8,2.4,4.1,3.7,2.60,4.2,2.5
3,April,3.8,3.1,2.4,4.4,2.4,3.8,3.7,2.50,3.6,2.9
4,May,3.5,3.2,1.9,3.4,2.7,2.7,3.9,2.20,4.5,3.9


In [90]:
months = roi_mat['Unnamed: 0'].tolist()
reinvested_amount = 0
month_allocation = {}
counter = 0
for m in months:
    MarkMonthMod=gp.Model()
    MarkMonthModX = MarkMonthMod.addMVar(10)
    MarkMonthMod.setObjective(gp.quicksum((roi_mat.iloc[counter][i+1]/100)*MarkMonthModX[i] for i in range(10)),sense=gp.GRB.MAXIMIZE)
    consls=[0]*3
    consls[0] = MarkMonthMod.addConstr(MarkMonthModX[0] + MarkMonthModX[1] <= MarkMonthModX[4] + MarkMonthModX[9])
    consls[1] = MarkMonthMod.addConstr(MarkMonthModX[4] + MarkMonthModX[5] + MarkMonthModX[6] + MarkMonthModX[7] + MarkMonthModX[8] >= 2*(MarkMonthModX[2] + MarkMonthModX[3]))
    consls[2] = MarkMonthMod.addConstr(gp.quicksum(MarkMonthModX[i]for i in range(10)) <= (10 + reinvested_amount)) 
    varimonth = MarkMonthMod.addConstrs((MarkMonthModX[i] <=3) for i in range(10))
    MarkMonthMod.Params.OutputFlag = 0
    MarkMonthMod.optimize()
    curr_obj = MarkMonthMod.objVal
    month_allocation[m] = list(MarkMonthModX.x)
    reinvested_amount = reinvested_amount + 0.5 * curr_obj
    counter +=1

df_all = pd.DataFrame(month_allocation)
channels = first_roi.index.to_list()

df_all = df_all.rename(index=dict(zip(df_all.index, channels)))
df_all

    
    

,January,February,March,April,May,June,July,August,September,October,November,December
Print,3.000000,3.0000,0.000000,0.000000,1.8041,3.000000,1.123777,3.000000,1.362933,0.000000,3.000000,3.000000
TV,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
SEO,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AdWords,1.333333,2.3955,3.000000,3.000000,0.0000,0.000000,3.000000,1.827294,3.000000,3.000000,2.056421,0.427951
Facebook,0.000000,3.0000,0.000000,0.000000,0.0000,0.000000,1.123777,0.000000,0.000000,0.000000,0.000000,3.000000
LinkedIn,0.000000,0.0000,3.000000,3.000000,0.0000,0.000000,0.000000,0.654588,3.000000,3.000000,1.112842,0.000000
Instagram,2.666667,0.0000,1.389648,3.000000,3.0000,3.000000,3.000000,0.000000,0.000000,3.000000,3.000000,0.000000
Snapchat,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Twitter,0.000000,1.7910,3.000000,1.596856,3.0000,2.020172,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000
Email,3.000000,0.0000,0.000000,0.000000,3.0000,3.000000,0.000000,3.000000,1.362933,2.955475,3.000000,3.000000


### Checking for Stable Month

In [92]:
subs = df_all.columns[1:]

# Iterate through the columns and perform subtraction
for col in subs:
    df_all[col] = df_all[col] - df_all[subs[subs.get_loc(col) - 1]]
    
df_all['January'] = 0
df_all


,January,February,March,April,May,June,July,August,September,October,November,December
Print,0,-1.709190,1.709190,-1.709190,3.513290,-2.317390,2.245267,0.826855,-2.536045,4.245235,-2.954425,4.663615
TV,0,-3.000000,6.000000,-9.000000,12.000000,-15.000000,18.000000,-21.000000,24.000000,-27.000000,30.000000,-30.000000
SEO,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AdWords,0,2.801176,-1.768726,3.736275,-5.703824,7.671374,-6.638923,7.433766,-5.228610,6.023453,-4.761876,3.928249
Facebook,0,-1.876223,1.876223,-1.876223,1.876223,-1.876223,3.000000,-4.123777,5.247555,-6.371332,7.495110,-5.618887
LinkedIn,0,0.458254,2.541746,-2.541746,2.541746,-2.541746,2.541746,-1.887158,4.232570,-3.577982,4.036236,-4.494489
Instagram,0,1.389648,0.000000,1.610352,-0.220704,1.831056,-0.441408,-0.948240,2.337888,-0.727536,2.117184,-3.506832
Snapchat,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Twitter,0,5.382972,-4.173972,4.561828,-1.949685,1.357713,2.234259,-2.826231,6.418203,-10.010174,13.602146,-17.194118
Email,0,-4.592543,7.592543,-10.592543,16.592543,-19.592543,22.592543,-22.592543,23.955475,-22.362933,23.770390,-22.177847


Allocations are not stable, we can imporve the model as follows:
- a
- b
